The aim of this code is to analyse trends in high-growth companies and understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. This project provides information that give companies a competitive insight as to industry trends and how they should structure their portfolio looking forward.

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


In [54]:
-- Looking at the top 10 companies with the highest valuation
SELECT industries.industry, companies.company, funding.valuation, funding.funding
FROM funding
INNER JOIN companies ON funding.company_id = companies.company_id
INNER JOIN industries ON funding.company_id = industries.company_id
ORDER BY funding.valuation DESC
LIMIT 10;

,industry,company,valuation,funding
0,Artificial intelligence,Bytedance,180000000000,8000000000
1,Other,SpaceX,100000000000,7000000000
2,E-commerce & direct-to-consumer,SHEIN,100000000000,2000000000
3,Fintech,Stripe,95000000000,2000000000
4,Fintech,Klarna,46000000000,4000000000
5,Internet software & services,Canva,40000000000,572000000
6,Fintech,Checkout.com,40000000000,2000000000
7,"Supply chain, logistics, & delivery",Instacart,39000000000,3000000000
8,Consumer & retail,JUUL Labs,38000000000,14000000000
9,Data management & analytics,Databricks,38000000000,3000000000


In [55]:
-- The number of unicorn and its valuation for each industry in 2019, 2020, and 2021
WITH unicorn_join_year AS (
	SELECT d.company_id, 
		   EXTRACT(year FROM d.date_joined) AS year_joined
	FROM dates AS d
	INNER JOIN industries AS i
		ON i.company_id = d.company_id
	WHERE EXTRACT(year FROM d.date_joined) IN (2019, 2020, 2021)
)
SELECT i.industry, 
	   ujy.year_joined AS year,
	   COUNT(f.valuation) AS num_unicorns, 
	   ROUND(AVG(f.valuation/1000000000), 2) AS average_valuation_billions
FROM industries AS i
INNER JOIN unicorn_join_year AS ujy 
	ON i.company_id = ujy.company_id
INNER JOIN funding AS f
	ON i.company_id = f.company_id
GROUP BY i.industry, ujy.year_joined
ORDER BY ujy.year_joined DESC, num_unicorns DESC;

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Health,2021,40,1.95
4,Artificial intelligence,2021,36,1.42
5,Cybersecurity,2021,27,2.52
6,"Supply chain, logistics, & delivery",2021,25,2.20
7,Data management & analytics,2021,21,2.14
8,Other,2021,21,1.71
9,Hardware,2021,14,2.00


In [62]:
-- Top 3 industries in 2019, 2020, 2021
WITH unicorn_join_year AS (
    SELECT d.company_id, 
           EXTRACT(year FROM d.date_joined) AS year_joined
    FROM dates AS d
    INNER JOIN industries AS i
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) IN (2019, 2020, 2021)
),
ranked_unicorns AS (
    SELECT i.industry, 
           ujy.year_joined AS year,
           COUNT(f.valuation) AS num_unicorns, 
           ROUND(AVG(f.valuation/1000000000), 2) AS average_valuation_billions,
           ROW_NUMBER() OVER (PARTITION BY ujy.year_joined ORDER BY COUNT(f.valuation) DESC) AS industry_rank
    FROM industries AS i
    INNER JOIN unicorn_join_year AS ujy 
        ON i.company_id = ujy.company_id
    INNER JOIN funding AS f
        ON i.company_id = f.company_id
    GROUP BY i.industry, ujy.year_joined
)
SELECT industry, year, num_unicorns, average_valuation_billions
FROM ranked_unicorns
WHERE industry_rank <= 3
ORDER BY year DESC, num_unicorns DESC;

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Artificial intelligence,2019,14,4.50
8,Internet software & services,2019,13,4.23
